# 코너점 검출

In [2]:
import cv2
import numpy as np

def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT,ksize=(11,11))
    dilate = cv2.dilate(src,kernel)
    localMax = (src==dilate)
    
    erode = cv2.erode(src,kernel)
    localMax2 = src>erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0,1]] = points[:,[1,0]]
    return points

src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray,ksize=3)
ret,res2 = cv2.threshold(np.abs(res),0.1,0,cv2.THRESH_TOZERO)

corners = findLocalMaxima(res2)
print('corners.shape=',corners.shape)

dst = src.copy()
for x,y in corners:
    cv2.circle(dst,(x,y),5,(0,0,255),2)
    
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

corners.shape= (8, 2)


In [2]:
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

res = cv2.cornerEigenValsAndVecs(gray,blockSize=5,ksize=3)
print('res.shape=',res.shape)
eigen = cv2.split(res)

T = 0.2
ret,edge = cv2.threshold(eigen[0],T,255,cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

corners = np.argwhere(eigen[1]>T)
corners[:,[0,1]] = corners[:,[1,0]]
print('len(corners=)',len(corners))

dst = src.copy()
for x,y in corners:
    cv2.circle(dst,(x,y),5,(0,0,255),2)
 
cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()
    

res.shape= (512, 512, 6)
len(corners=) 8


In [3]:
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

eigen = cv2.cornerMinEigenVal(gray,blockSize=5)
print('eigen.shape=',eigen.shape)

T = 0.2
corners = np.argwhere(eigen > T)
corners[:,[0,1]] = corners[:,[1,0]]
print('len(corners)=',len(corners))
dst = src.copy()

for x,y, in corners:
    cv2.circle(dst,(x,y),3,(0,0,255),2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()    

eigen.shape= (512, 512)
len(corners)= 8


In [7]:
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT,ksize=(11,11))
    dilate = cv2.dilate(src,kernel)
    localMax = (src==dilate)
    
    erode = cv2.erode(src,kernel)
    localMax2 = src>erode
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0,1]] = points[:,[1,0]]
    return points

src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray,blockSize=5,ksize=3,k=0.01)
ret,res = cv2.threshold(np.abs(res),0.02,0,cv2.THRESH_TOZERO)

res8 = cv2.normalize(res,None,0,255,cv2.NORM_MINMAX,dtype=cv2.CV_8U)

corners = findLocalMaxima(res)
print('corners=',corners)

corners = corners.astype(np.float32,order='C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS,10,0.01)
corners2 = cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),term_crit)
print('corners2',corners2)

dst = src.copy()
for x,y in np.int32(corners2):
    cv2.circle(dst,(x,y),3,(0,0,255),2)

cv2.imshow('res8', res8)    
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()        

corners= [[101 101]
 [249 101]
 [252 168]
 [349 171]
 [101 249]
 [248 252]
 [251 349]
 [349 349]]
corners2 [[ 99.55941  99.5577 ]
 [250.44127  99.5592 ]
 [250.5585  169.43939]
 [350.43945 169.56058]
 [ 99.55909 250.43958]
 [249.43884 250.55685]
 [249.56105 350.4409 ]
 [350.44147 350.44037]]


In [11]:
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

K=5
corners = cv2.goodFeaturesToTrack(gray,maxCorners=K,qualityLevel=0.05,minDistance=10)
print('corners.shape=',corners.shape)
print('corners=',corners)

K=10
corners2 = cv2.goodFeaturesToTrack(gray,maxCorners=K,qualityLevel=0.05,
                                   minDistance=10, useHarrisDetector = True, k = 0.04)
print('corners2.shape=',corners2.shape)
print('corners2=',corners2)

dst = src.copy()
dst2 = src.copy()
corners = corners.reshape(-1,2)
for x,y in corners:
    cv2.circle(dst,(int(x),int(y)),5,(0,0,255),-1)
    
corners2 = corners2.reshape(-1,2)
for x,y in corners2:
    cv2.circle(dst2,(int(x),int(y)),5,(0,0,255),-1)

cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()     

corners.shape= (5, 1, 2)
corners= [[[350. 350.]]

 [[250. 350.]]

 [[249. 251.]]

 [[100. 100.]]

 [[100. 250.]]]
corners2.shape= (8, 1, 2)
corners2= [[[350. 350.]]

 [[100. 100.]]

 [[249. 251.]]

 [[100. 250.]]

 [[250. 350.]]

 [[250. 100.]]

 [[350. 170.]]

 [[251. 169.]]]


# 체스보드 패턴 코너점 검출

In [12]:
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

# 책에서는 (6,3)
patternSize = (7,6)
found,corners = cv2.findChessboardCorners(src,patternSize)
print('corners.shape=',corners.shape)

term_crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,10,0.01)
corners2 = cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst,patternSize,corners2,found)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()  

corners.shape= (42, 1, 2)


In [13]:
src = cv2.imread('./data/circleGrid.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
patternSize = (4,4)
found,centers = cv2.findCirclesGrid(src,patternSize)
print('centers.shape=',centers.shape)

term_crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,10,0.01)
centers2 = cv2.cornerSubPix(gray,centers,(5,5),(-1,-1),term_crit)

dst = src.copy()
cv2.drawChessboardCorners(dst,patternSize,centers2,found)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()  

centers.shape= (16, 1, 2)


# 모멘트

In [3]:
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)

# 모멘트를 구하기 위해서는 grayscale image를 binary image로 바꾸는 과정이 필요함
ret,bimage = cv2.threshold(gray,128,255,cv2.THRESH_BINARY)

M = cv2.moments(bimage,True)
for key,value in M.items():
    print('{}={}'.format(key,value))
    
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
dst = src.copy()
cv2.circle(dst,(cx,cy),10,(0,0,255),-1)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()  

m00=41001.0
m10=9454225.0
m01=8726225.0
m20=2397368875.0
m11=2119946125.0
m02=2023688875.0
m30=651249446875.0
m21=560532817875.0
m12=514587165875.0
m03=504399486875.0
mu20=217364232.4150629
mu11=107807529.58464432
mu02=166490203.00114632
mu30=-1790368560.1228027
mu21=584194816.4979553
mu12=2065142657.361374
mu03=2830588638.024536
nu20=0.1293001961152179
nu11=0.06412984584958643
nu02=0.09903752636820143
nu30=-0.00525964363794542
nu21=0.0017162145372477983
nu12=0.006066859461883883
nu03=0.008315543432356281


In [4]:
src = cv2.imread('./data/circle2.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
ret,bimage = cv2.threshold(gray,128,255,cv2.THRESH_BINARY_INV)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,hierarchy = cv2.findContours(bimage,mode,method)

dst = src.copy()
cv2.drawContours(dst,contours,-1,(255,0,0),3)

for cnt in contours:
    M = cv2.moments(cnt,True)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.circle(dst,(cx,cy),5,(0,0,255),2)
    
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()     

In [5]:
# hu 모멘트: 이동, 스케일, 회전에 불변이다.

src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
ret,bimage = cv2.threshold(gray,128,255,cv2.THRESH_BINARY)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,hierarchy = cv2.findContours(bimage,mode,method)

dst = src.copy()
cnt = contours[0]
cv2.drawContours(dst,[cnt],0,(255,0,0),3)

M = cv2.moments(cnt)
hu = cv2.HuMoments(M)
print('hu.shape=',hu.shape)
print('hu=',hu)

angle = 45.0
scale = 0.2
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
center = (cx,cy)
t = (20,30)
A = cv2.getRotationMatrix2D(center,angle,scale)
A[:,2] +=t
print('A=',A)
cnt2 = cv2.transform(cnt,A)
cv2.drawContours(dst,[cnt2],0,(0,255,0),3)
cv2.imshow('dst', dst)

M2 = cv2.moments(cnt)
hu2 = cv2.HuMoments(M2)
print('hu2.shape=',hu2.shape)
print('hu2=',hu2)

# hu와 hu2의 차이는 거의 없다.
diffSum = np.sum(cv2.absdiff(hu,hu2))
print('diffSum=',diffSum)

cv2.waitKey()
cv2.destroyAllWindows()  

hu.shape= (7, 1)
hu= [[ 2.29512832e-01]
 [ 1.77264199e-02]
 [ 5.73565197e-04]
 [ 1.05422872e-04]
 [ 9.93149152e-09]
 [-7.94530682e-07]
 [-2.39456345e-08]]
A= [[ 1.41421356e-01  1.41421356e-01  1.87491761e+02]
 [-1.41421356e-01  1.41421356e-01  2.44545584e+02]]
hu2.shape= (7, 1)
hu2= [[ 2.29512832e-01]
 [ 1.77264199e-02]
 [ 5.73565197e-04]
 [ 1.05422872e-04]
 [ 9.93149152e-09]
 [-7.94530682e-07]
 [-2.39456345e-08]]
diffSum= 0.0


# 모양 관련 특징 검출

In [7]:
src = cv2.imread('./data/banana.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
src = cv2.cvtColor(src,cv2.COLOR_BGR2RGB)
ret,bimage = cv2.threshold(gray,220,255,cv2.THRESH_BINARY_INV)
bimage = cv2.dilate(bimage,None)
cv2.imshow('bImage', bimage)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,hierarchy = cv2.findContours(bimage,mode,method)
print('len(contorus)=',len(contours))

maxLength=0
k=0
for i,cnt in enumerate(contours):
    perimeter = cv2.arcLength(cnt,closed=True)
    if perimeter > maxLength:
        maxLength = perimeter
        k=i
print('maxLength=',maxLength)
cnt = contours[k]
dst2 = src.copy()
cv2.drawContours(dst2, [cnt], 0, (255, 0, 0), 3)
cv2.imshow('dst2', dst2)

area = cv2.contourArea(cnt)
print('area=',area)
x,y,width,height = cv2.boundingRect(cnt)
dst3 = dst2.copy()
cv2.rectangle(dst3,(x,y),(x+width,y+height),(0,0,255),2)
cv2.imshow('dst3', dst3)

rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int32(box)
print('box=',box)
dst4 = dst2.copy()
cv2.drawContours(dst4,[box],0,(0,0,255),2)
cv2.imshow('dst4', dst4)

(x,y),radius = cv2.minEnclosingCircle(cnt)
dst5 = dst2.copy()
cv2.circle(dst5,(int(x),int(y)),int(radius),(0,0,255),2)
cv2.imshow('dst5', dst5)

cv2.waitKey()
cv2.destroyAllWindows()  

len(contorus)= 7
maxLength= 2042.1677330732346
area= 48305.5
box= [[ 76 496]
 [447  32]
 [636 183]
 [265 647]]


In [10]:
src = cv2.imread('./data/hand.jpg')
hsv = cv2.cvtColor(src,cv2.COLOR_BGR2HSV)
lowerb = (0,40,0)
upperb = (20,180,255)
bimage = cv2.inRange(hsv,lowerb,upperb)
cv2.imshow('bimage', bimage)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,hierachy = cv2.findContours(bimage,mode,method)

dst = src.copy()
cnt = contours[0]
cv2.drawContours(dst,[cnt],0,(255,0,0),2)

dst2 = dst.copy()
rows,cols = dst2.shape[:2]
hull = cv2.convexHull(cnt)
cv2.drawContours(dst2,[hull],0,(0,0,255),2)

cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()  

In [17]:
src = cv2.imread('/Users/yongchanchun/Desktop/데스크탑 - MacBook Pro/computer vision/data/hand.jpg')
hsv = cv2.cvtColor(src,cv2.COLOR_BGR2HSV)
lowerb = (0,40,0)
upperb = (20,180,255)
bimage = cv2.inRange(hsv,lowerb,upperb)
cv2.imshow('bimage', bimage)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,hierachy = cv2.findContours(bimage,mode,method)

dst = src.copy()
cnt = contours[0]
cv2.drawContours(dst,[cnt],0,(255,0,0),2)

dst2 = dst.copy()
rows,cols = dst2.shape[:2]
hull = cv2.convexHull(cnt, returnPoints = False)
hull_points = cnt[hull[:,0]]
cv2.drawContours(dst2,[hull_points],0,(255,0,255),6)

T = 5
defects = cv2.convexityDefects(cnt,hull)
# print('defects.shape=',defects.shape)

for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    dist = d/256
    start = tuple(cnt[s][0])
    end = tuple(cnt[e][0])
    far = tuple(cnt[f][0])
    
    if dist> T:
        cv2.line(dst2,start,end,[255,255,0],2)
        cv2.line(dst2,start,far,[255,255,0],1)
        cv2.line(dst2,end,far,[255,255,0],1)
        
        cv2.circle(dst2,start,5,[0,255,255],-1)
        cv2.circle(dst2,end,5,[0,128,255],-1)
        cv2.circle(dst2,far,5,[0,0,255],-1)
 

cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()         

AttributeError: 'NoneType' object has no attribute 'shape'

# 모양 매칭

In [31]:
ref_src = cv2.imread('./data/refShapes1.jpg')
ref_gray = cv2.cvtColor(ref_src,cv2.COLOR_BGR2GRAY)
ret,ref_bin = cv2.threshold(ref_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

test_src = cv2.imread('./data/refShapes.jpg')
test_gray = cv2.cvtColor(test_src,cv2.COLOR_BGR2GRAY)
ret,test_bin = cv2.threshold(test_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
ref_contours,_ = cv2.findContours(ref_bin,mode,method)
test_contours,_ = cv2.findContours(test_bin1,mode,method)

ref_dst = ref_src.copy()
colors = ((0, 0, 255), (0, 255, 0), (255, 0, 0))
for i, cnt in enumerate(ref_contours):
    cv2.drawContours(ref_dst, [cnt], 0, colors[i], 2)

test_dst = test_src.copy()
method = cv2.CONTOURS_MATCH_I1
for i, cnt1 in enumerate(test_contours):
    matches = []
    for cnt2 in ref_contours:
        ret = cv2.matchShapes(cnt1, cnt2, method, 0)
        matches.append(ret)
    k = np.argmin(matches)
    cv2.drawContours(test_dst, [cnt1], 0, colors[k], 2)

cv2.imshow('ref_dst', ref_dst)
cv2.imshow('test_dst', test_dst)
cv2.waitKey()
cv2.destroyAllWindows()   


# 적분영상

In [32]:
A = np.arange(1,17).reshape(4,4).astype(np.uint8)
print('A=',A)

sumA,sqsumA,tiltedA = cv2.integral3(A)
print('sumA=',sumA)
print('sqsumA=',sqsumA)
print('tiltedA=',tiltedA)

A= [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
sumA= [[  0   0   0   0   0]
 [  0   1   3   6  10]
 [  0   6  14  24  36]
 [  0  15  33  54  78]
 [  0  28  60  96 136]]
sqsumA= [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.000e+00 5.000e+00 1.400e+01 3.000e+01]
 [0.000e+00 2.600e+01 6.600e+01 1.240e+02 2.040e+02]
 [0.000e+00 1.070e+02 2.470e+02 4.260e+02 6.500e+02]
 [0.000e+00 2.760e+02 6.120e+02 1.016e+03 1.496e+03]]
tiltedA= [[ 0  0  0  0  0]
 [ 0  1  2  3  4]
 [ 1  8 12 16 15]
 [ 8 26 38 42 36]
 [26 60 80 84 70]]


In [ ]:
gray = cv2.imread('./data/lena.jpg',0)
gray_sum = cv2.integral(gray)
dst = cv2.normalize(gray_sum,None,0,255,cv2.NORM_MINMAX,dtype=cv2.CV_8U)
dst = cv2.cvtColor(dst,cv2.COLOR_BGR2RGB)

cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows() 